#Install Necessary Packages

In [1]:
# !pip install  langchain langchain-community langchain-openai langchain-experimental wikipedia tiktoken py2neo streamlit neo4j langchain-core
#!pip install yfiles_jupyter_graphs --quiet
# !pip install yfiles_jupyter_graphs_for_neo4j --quiet
# from yfiles_jupyter_graphs import GraphWidget

# Load Packages

## Run Liabraries

In [2]:
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer


## Authenticate Online Version

In [3]:
# Using the online version of Neo4j as colab had issue connecting to desktop version
from langchain_community.graphs import Neo4jGraph
import os
from py2neo import Graph
os.environ["OPENAI_API_KEY"] = "OPENAI_KEY"

# No need to set these if you use the explicit parameters below
try:
    graph = Graph(uri="bolt://localhost:7687", auth=("neo4j", "12345678"))
except ValueError as e:
    print(f"Error connecting to Neo4j: {e}")
else:
    print("Successfully connected to Neo4j!")
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="12345678")

Successfully connected to Neo4j!


## Using Langchain Data Loader

### Load Test Data

In [4]:
import PyPDF2
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Neo4jVector

pdf_file = "Financial_report.pdf"

# Extract text from PDF using pdfminer.six
from pdfminer.high_level import extract_text
text = extract_text(pdf_file)

# Write text to a file
text_path = "extracted_text.txt"
with open(text_path, 'w', encoding='utf-8') as text_file:
    text_file.write(text)

# Load text from the file
loader = TextLoader(text_path)
documents = loader.load()


# from langchain.document_loaders import TextLoader
# text_loader = TextLoader("Code for Graphs.txt")  # Replace with your file path
# documents = text_loader.load()
# documents[0]

In [5]:
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=80, chunk_overlap=10)
documents = text_splitter.split_documents(documents[:3])

Now it's time to construct a graph based on the retrieved documents. For this purpose, we have implemented an `LLMGraphTransformermodule` that significantly simplifies constructing and storing a knowledge graph in a graph database.

## Initiate the LLM

1. yfiles_jupyter_graphs only supports currently function calling models from OpenAI and Mistral

In [6]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

langchain_graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{langchain_graph_documents [0].nodes}")
print(f"Relationships:{langchain_graph_documents [0].relationships}")

In [ ]:
for nodes in langchain_graph_documents [0].nodes:
  print(nodes)

In [ ]:
from py2neo import Graph, Node  # Import Node explicitly

In [ ]:
print(langchain_graph_documents [0].nodes[0])  # Inspect the first node


In [ ]:
langchain_graph_documents [0].relationships

In [ ]:
print(type(langchain_graph_documents ))  # Check the type of the graph object

In [ ]:
py2neo_graph = Graph(uri="bolt://localhost:7687", auth=("neo4j", "12345678"))

In [ ]:
for node in langchain_graph_documents[0].nodes:
    neo4j_node = Node("Concept", name=node.id, content='')
    py2neo_graph.create(neo4j_node)

In [ ]:
langchain_graph_documents[0].relationships

In [ ]:
from pyvis.network import Network
from py2neo import Graph

# Your Neo4j connection setup
py2neo_graph = Graph(uri="bolt://localhost:7687", auth=("neo4j", "12345678"))

net = Network(height="600px", width="100%", directed=True)

# Add all nodes from the graph documents (you might want to filter if there are many)
for node in langchain_graph_documents[0].nodes:
    net.add_node(node.id, label=node.id, title=node.type)

# Add relationships
for relationship in langchain_graph_documents[0].relationships:
    net.add_edge(relationship.source.id, relationship.target.id,
                 label=relationship.type, title=relationship.type)

net.save_graph("mygraph1.html")  # Save the graph
# net.show("mygraph.html") # dont run this in colab as throws error of rendering , hence i had to save it as a graph

In [ ]:
from IPython.display import IFrame

IFrame(src='/content/mygraph.html', width=800, height=600) # Adjust 'width' and 'height' as needed
# this code has some issue to load the file in colab

In [ ]:
# !pip install pdfminer.six

import PyPDF2
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Neo4jVector

pdf_file = "Financial_report.pdf"

# Extract text from PDF using pdfminer.six
from pdfminer.high_level import extract_text
text = extract_text(pdf_file)

# Write text to a file
text_path = "extracted_text.txt"
with open(text_path, 'w', encoding='utf-8') as text_file:
    text_file.write(text)

# Load text from the file
loader = TextLoader(text_path)
documents = loader.load()

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(OPENAI_API_KEY="sk-proj-23bracs3hqg4VfQS8nZiT3BlbkFJ3NBRNwm2MFcrGolCRVYy")

# Connect to Neo4j and load documents
db = Neo4jVector.from_documents(
    docs, OpenAIEmbeddings(), url="bolt://localhost:7687", username="neo4j", password="12345678"
)
 
query = "What are AI/ML applications are designed to improve product quality?"
docs_with_score = db.similarity_search_with_score(query, k=2)
print(docs_with_score)


In [ ]:
# from google.colab import files

# files.download('mygraph.html')
# this works fine

# New Data - CSV Coding